In [391]:
import sys
import pandas as pd
import logging, datetime, os
from sklearn.cluster import KMeans
import warnings
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedShuffleSplit, StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#%config Application.log_level="INFO"
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.simplefilter('ignore')

## Файлы

In [257]:
# клиенты с одним продуктом (выгрузка из базы)
#file_name = 'clients_clustering.csv'

file_name = 'INSURE_TEXT_MINING/data_insure_2018.csv'

# клиенты с несколькими продуктами (результат классификации платежек)
#file_name = 'multiproduct_clients_clustering.csv'

# клиенты с одним продуктом (результат классификации платежек)
#file_name = 'one_product_clients_clustering.csv'

## Формирование исходных данных с суммами

In [388]:
# Инициализация исходных даных:
    
data = pd.read_csv(file_name, encoding='cp1251', delimiter=';')
data = data.rename(columns = {"PREDICTED" : "CLASS"})
data = data.dropna(subset=['CLASS'])

# Список продуктов

labels = sorted(list(data['CLASS'].drop_duplicates()))

# Формируем данные для кластеризации (ИНН и колечество платежей по продуктам)

data['COUNT'] = 1
data['SUMMA'] = pd.to_numeric(data['SUMMA'].apply(lambda x: x.replace(',','.')))
data['SUMMA_MEAN'] = data['SUMMA']
data['SUMMA_STD'] = data['SUMMA']

data_pivot = pd.pivot_table(data,
               values=[
                        'SUMMA',
                        'SUMMA_MEAN',                       
                        'SUMMA_STD',                       
                        'COUNT',
                      ],
               index = ['INN','CLASS'],
               aggfunc={
                   'SUMMA' : np.sum,
                   'SUMMA_MEAN' : np.mean,
                   'SUMMA_STD' : np.std,
                   'COUNT' : np.sum,
               }).unstack().fillna(0)
data_pivot['INN'] = data_pivot.index

# Загружаем размеченных клиентов

file_2_name = 'inn_product.csv'

data_label = pd.read_csv(file_2_name, encoding='cp1251', delimiter=';')
data_label = data_label.drop_duplicates()
data_label = data_label.rename(columns = {"PARTICIPANT_INN" : "INN"})

# Витрина по всем клиентам (ИНН и количество платежей по продуктам и класс клиента)

data_merged = pd.merge(data_pivot,data_label,on='INN')

# Витрина по клиентам, имеющим только один продукт (ИНН и количество платежей по продуктам и класс клиента)

data_label_counts = data_label.groupby('INN',as_index = False).count()
inn_with_1_product = pd.DataFrame(data_label_counts[data_label_counts['CLASS']  == 1]['INN'].drop_duplicates())
data_label_inn_with_1_product = pd.merge(data_label,inn_with_1_product,on='INN')

data_merged_inn_with_1_product = pd.merge(data_pivot,data_label_inn_with_1_product,on='INN')

labels_for_training = [x for x in list(data_merged_inn_with_1_product.columns) if 'INN' not in x and 'CLASS' not in x]

# Нормализуем численные значения

data_values = data_merged_inn_with_1_product[labels_for_training].values
scaler = preprocessing.Normalizer()
standardscaler = preprocessing.StandardScaler()
data_merged_inn_with_1_product[labels_for_training] = standardscaler.fit_transform(scaler.fit_transform(data_values))

## Для клиентов с одним продуктом

In [403]:
X_train1,X_test1,y_train1,y_test1 = train_test_split(data_merged_inn_with_1_product[labels_for_training],
                                                     data_merged_inn_with_1_product['CLASS'],
                                                     test_size = 0.2,
                                                     random_state = 40
                                                     )
text_clf_xgb = XGBClassifier()
text_clf_xgb = text_clf_xgb.fit(X_train1,y_train1)
predicted_values_xgb = text_clf_xgb.predict(X_test1)
print(metrics.classification_report(y_test1,
                                    predicted_values_xgb,
                                    target_names = data_merged_inn_with_1_product['CLASS'].unique()))

                                                        precision    recall  f1-score   support

                                               Питание       1.00      0.95      0.98        22
                                Строительные материалы       1.00      1.00      1.00        24
                                                Ремонт       1.00      0.67      0.80         3
                                           Медицинский       1.00      0.67      0.80         3
                                                Охрана       0.95      0.97      0.96        62
                                                   ГСМ       1.00      0.88      0.93        16
                                         Хозяйственный       0.67      0.50      0.57         4
Оказание услуг по проведению лабораторных исследований       1.00      0.80      0.89         5
                                            Автомобили       0.97      0.95      0.96        60
                                       

## Для клиентов с несколькими продуктами

In [315]:
X_train,X_test,y_train,y_test = train_test_split(data_merged[labels_for_training],
                                                 data_merged['CLASS'],
                                                 test_size = 0.2,
                                                 random_state = 40)
text_clf_xgb = XGBClassifier()
text_clf_xgb = text_clf_xgb.fit(X_train,y_train)
predicted_values_xgb = text_clf_xgb.predict(X_test)
print(metrics.classification_report(y_test,
                                    predicted_values_xgb,
                                    target_names = data_merged['CLASS'].unique()))

                                                        precision    recall  f1-score   support

                                               Питание       0.89      0.89      0.89        18
                                Строительные материалы       0.96      1.00      0.98        26
                                                Ремонт       1.00      1.00      1.00         1
                                           Медицинский       0.00      0.00      0.00         3
                                                Охрана       0.99      0.99      0.99        76
                                                   ГСМ       0.67      1.00      0.80        10
                                         Хозяйственный       0.33      0.50      0.40         2
                                         Электротовары       0.88      1.00      0.93         7
Оказание услуг по проведению лабораторных исследований       0.92      0.96      0.94        46
                                       

## Смотрим сколько клиентов в триггерах

In [261]:
from loader import Loader
import os, sys

        
file= 'tmp_select_from_triggers.csv'
query = 'select distinct inn from atb_segmen_tr_all_t'
print(query)

data=Loader(True).save_csv(query, path=file, verbose=1)   
data_triggers = pd.read_csv('tmp_select_from_triggers.csv', encoding='utf-8', delimiter=';')
data_triggers.columns = ['INN']

select distinct inn from atb_segmen_tr_all_t
Connecting...
Getting data ... 
Downloaded 10,000 lines,       0sec. passed
Downloaded 20,000 lines,       0sec. passed
Downloaded 30,000 lines,       0sec. passed
Downloaded 40,000 lines,       0sec. passed
Downloaded 50,000 lines,       0sec. passed
Downloaded 60,000 lines,       0sec. passed
Downloaded 70,000 lines,       0sec. passed
Downloaded 80,000 lines,       0sec. passed
Downloaded 90,000 lines,       0sec. passed
Downloaded 100,000 lines,       0sec. passed
Downloaded 110,000 lines,       0sec. passed
Downloaded 120,000 lines,       0sec. passed
Downloaded 130,000 lines,       0sec. passed
Downloaded 140,000 lines,       0sec. passed
Downloaded 150,000 lines,       0sec. passed
Downloaded 160,000 lines,       0sec. passed
Downloaded 170,000 lines,       0sec. passed
Downloaded 180,000 lines,       0sec. passed
Downloaded 190,000 lines,       0sec. passed
Downloaded 200,000 lines,       0sec. passed
Downloaded 210,000 lines,       

Downloaded 1,790,000 lines,       3sec. passed
Downloaded 1,800,000 lines,       3sec. passed
Downloaded 1,810,000 lines,       3sec. passed
Downloaded 1,820,000 lines,       3sec. passed
Downloaded 1,830,000 lines,       3sec. passed
Downloaded 1,840,000 lines,       3sec. passed
Downloaded 1,850,000 lines,       4sec. passed
Downloaded 1,860,000 lines,       4sec. passed
Downloaded 1,870,000 lines,       4sec. passed
Downloaded 1,880,000 lines,       4sec. passed
Downloaded 1,890,000 lines,       4sec. passed
Downloaded 1,900,000 lines,       4sec. passed
Downloaded 1,910,000 lines,       4sec. passed
Downloaded 1,920,000 lines,       4sec. passed
Downloaded 1,930,000 lines,       4sec. passed
Downloaded 1,940,000 lines,       4sec. passed
Downloaded 1,950,000 lines,       4sec. passed
Downloaded 1,960,000 lines,       4sec. passed
Downloaded 1,970,000 lines,       4sec. passed
Downloaded 1,980,000 lines,       4sec. passed
Downloaded 1,990,000 lines,       4sec. passed
Downloaded 2,

In [279]:
len(pd.merge(data_triggers,data,on = 'INN')['INN'].drop_duplicates())

2163

In [280]:
len(data['INN'].drop_duplicates())

2167